In [ ]:
# to handle datasets
import pandas as pd
import numpy as np

# for plotting
import matplotlib.pyplot as plt

# to display all the columns of the dataframe in the notebook
pd.pandas.set_option('display.max_columns', None)

### Step 1. Gathering Data

In [ ]:
#directory
import os
path = "/kaggle/input/house-prices-advanced-regression-techniques"
os.chdir(path)

# load dataset
data = pd.read_csv('train.csv')

# rows and columns of the data
print(data.shape)
# visualise the dataset
data.head()

### Step 2. Preparing Data

In [ ]:
# make a list of the variables that contain missing values
vars_with_na = [var for var in data.columns if data[var].isnull().sum() > 0]

# determine percentage of missing values
data[vars_with_na].isnull().mean()

In [ ]:
data[vars_with_na].head()

In [ ]:
def analyse_na_value(df, var):
    df = df.copy()
    # let's make a variable that indicates 1 if the observation was missing or zero otherwise
    df[var] = np.where(df[var].isnull(), 1, 0)
    # let's compare the median SalePrice in the observations where data is missing
    # vs the observations where a value is available
    df.groupby(var)['SalePrice'].median().plot.bar()
    plt.title(var)
    plt.show()


# let's run the function on each variable with missing data
for var in vars_with_na:
    analyse_na_value(data, var)

In [ ]:
# make list of numerical variables
num_vars = [var for var in data.columns if data[var].dtypes != 'O']
print('Number of numerical variables: ', len(num_vars))

# visualise the numerical variables
data[num_vars].head()

In [ ]:
print('Number of House Id labels: ', len(data.Id.unique()))
print('Number of Houses in the Dataset: ', len(data))

In [ ]:
# list of variables that contain year information
year_vars = [var for var in num_vars if 'Yr' in var or 'Year' in var]
year_vars

In [ ]:
# let's explore the relationship between the year variables
# and the house price in a bit of more detail:
def analyse_year_vars(df, var):
    df = df.copy()
    # capture difference between year variable and year
    # in which the house was sold
    df[var] = df['YrSold'] - df[var]
    plt.scatter(df[var], df['SalePrice'])
    plt.ylabel('SalePrice')
    plt.xlabel(var)
    plt.show()
    
for var in year_vars:
    if var !='YrSold':
        analyse_year_vars(data, var)

In [ ]:
#  let's male a list of discrete variables
discrete_vars = [var for var in num_vars if len(
    data[var].unique()) < 20 and var not in year_vars+['Id']]
print('Number of discrete variables: ', len(discrete_vars))

In [ ]:
# let's visualise the discrete variables
data[discrete_vars].head()

In [ ]:
def analyse_discrete(df, var):
    df = df.copy()
    df.groupby(var)['SalePrice'].median().plot.bar()
    plt.title(var)
    plt.ylabel('Median SalePrice')
    plt.show()
    
for var in discrete_vars:
    analyse_discrete(data, var)

In [ ]:
# make list of continuous variables
cont_vars = [
    var for var in num_vars if var not in discrete_vars+year_vars+['Id']]
print('Number of continuous variables: ', len(cont_vars))

In [ ]:
# let's visualise the continuous variables
data[cont_vars].head()

In [ ]:
# Let's go ahead and analyse the distributions of these variables
def analyse_continuous(df, var):
    df = df.copy()
    df[var].hist(bins=30)
    plt.ylabel('Number of houses')
    plt.xlabel(var)
    plt.title(var)
    plt.show()

for var in cont_vars:
    analyse_continuous(data, var)

In [ ]:
# Let's go ahead and analyse the distributions of these variables
# after applying a logarithmic transformation
def analyse_transformed_continuous(df, var):
    df = df.copy()
    # log does not take 0 or negative values, so let's be
    # careful and skip those variables
    if any(data[var] <= 0):
        pass
    else:
        # log transform the variable
        df[var] = np.log(df[var])
        df[var].hist(bins=30)
        plt.ylabel('Number of houses')
        plt.xlabel(var)
        plt.title(var)
        plt.show()

for var in cont_vars:
    analyse_transformed_continuous(data, var)

In [ ]:
# let's explore the relationship between the house price and
# the transformed variables with more detail:
def transform_analyse_continuous(df, var):
    df = df.copy()
    # log does not take negative values, so let's be careful and skip those variables
    if any(data[var] <= 0):
        pass
    else:
        # log transform the variable
        df[var] = np.log(df[var])
        # log transform the target (remember it was also skewed)
        df['SalePrice'] = np.log(df['SalePrice'])
    
        # plot
        plt.scatter(df[var], df['SalePrice'])
        plt.ylabel('SalePrice')
        plt.xlabel(var)
        plt.show()


for var in cont_vars:
    if var != 'SalePrice':
        transform_analyse_continuous(data, var)

In [ ]:
# let's make boxplots to visualise outliers in the continuous variables
def find_outliers(df, var):
    df = df.copy()
    # log does not take negative values, so let's be
    # careful and skip those variables
    if any(data[var] <= 0):
        pass
    else:
        df[var] = np.log(df[var])
        df.boxplot(column=var)
        plt.title(var)
        plt.ylabel(var)
        plt.show()


for var in cont_vars:
    find_outliers(data, var)

In [ ]:
# capture categorical variables in a list
cat_vars = [var for var in data.columns if data[var].dtypes == 'O']
print('Number of categorical variables: ', len(cat_vars))

In [ ]:
# let's visualise the values of the categorical variables
data[cat_vars].head()

In [ ]:
data[cat_vars].nunique()

In [ ]:
def analyse_rare_labels(df, var, rare_perc):
    df = df.copy()
    tmp = df.groupby(var)['SalePrice'].count() / len(df)
    return tmp[tmp < rare_perc]

for var in cat_vars:
    print(analyse_rare_labels(data, var, 0.01))
    print()

In [ ]:
for var in cat_vars:
    # we can re-use the function to determine median
    # sale price, that we created for discrete variables
    analyse_discrete(data, var)

### Feature Engineering

#### Separating the data into train and test involves randomness, therefore, we need to set the seed.

In [ ]:
# to divide train and test set
from sklearn.model_selection import train_test_split

train_X, val_X, train_y, val_y = train_test_split(data,
                                                    data['SalePrice'],
                                                    test_size=0.1,
                                                    # we are setting the seed here:
                                                    random_state=0)  

train_X.shape, val_X.shape

#### Missing values

In [ ]:
# make a list of the categorical variables that contain missing values

vars_with_na = [
    var for var in data.columns
    if train_X[var].isnull().sum() > 0 and train_X[var].dtypes == 'O'
]

# print percentage of missing values per variable
train_X[vars_with_na].isnull().mean()

In [ ]:
# replace missing values with new label: "Missing"
train_X[vars_with_na] = train_X[vars_with_na].fillna('Missing')
val_X[vars_with_na] = val_X[vars_with_na].fillna('Missing')

In [ ]:
# check that we have no missing information in the engineered variables
train_X[vars_with_na].isnull().sum()

In [ ]:
# check that test set does not contain null values in the engineered variables
[var for var in vars_with_na if val_X[var].isnull().sum() > 0]

#### Numerical variables

In [ ]:
# make a list with the numerical variables that contain missing values
vars_with_na = [
    var for var in data.columns
    if train_X[var].isnull().sum() > 0 and train_X[var].dtypes != 'O'
]

# print percentage of missing values per variable
train_X[vars_with_na].isnull().mean()

In [ ]:
# replace engineer missing values as we described above
for var in vars_with_na:

    # calculate the mode using the train set
    mode_val = train_X[var].mode()[0]

    # add binary missing indicator (in train and test)
    train_X[var+'_na'] = np.where(train_X[var].isnull(), 1, 0)
    val_X[var+'_na'] = np.where(val_X[var].isnull(), 1, 0)

    # replace missing values by the mode
    # (in train and test)
    train_X[var] = train_X[var].fillna(mode_val)
    val_X[var] = val_X[var].fillna(mode_val)

# check that we have no more missing values in the engineered variables
train_X[vars_with_na].isnull().sum()

In [ ]:
# check that test set does not contain null values in the engineered variables
[var for var in vars_with_na if val_X[var].isnull().sum() > 0]

In [ ]:
# check the binary missing indicator variables
train_X[['LotFrontage_na', 'MasVnrArea_na', 'GarageYrBlt_na']].head()

#### Temporal variables

Capture elapsed time: there are 4 variables that refer to the years in which the house or the garage were built or remodeled. I captured the time elapsed between those variables and the year in which the house was sold.

In [ ]:
def elapsed_years(df, var):
    # capture difference between the year variable
    # and the year in which the house was sold
    df[var] = df['YrSold'] - df[var]
    return df

In [ ]:
for var in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    train_X = elapsed_years(train_X, var)
    val_X = elapsed_years(val_X, var)

#### Numerical variable transformation

I used log transformation on the positive numerical variables in order to get a more Gaussian-like distribution. This tends to help linear machine learning models.

In [ ]:
for var in ['LotFrontage', 'LotArea', '1stFlrSF', 'GrLivArea', 'SalePrice']:
    train_X[var] = np.log(train_X[var])
    val_X[var] = np.log(val_X[var])

In [ ]:
# check that test set does not contain null values in the engineered variables
[var for var in ['LotFrontage', 'LotArea', '1stFlrSF',
                 'GrLivArea', 'SalePrice'] if val_X[var].isnull().sum() > 0]

In [ ]:
# same for train set
[var for var in ['LotFrontage', 'LotArea', '1stFlrSF',
                 'GrLivArea', 'SalePrice'] if train_X[var].isnull().sum() > 0]

#### Categorical variables

##### Removing rare labels

First, I group those categories within variables that are present in less than 1% of the observations. That is, all values of categorical variables that are shared by less than 1% of houses, well be replaced by the string "Rare".

In [ ]:
# let's capture the categorical variables in a list
cat_vars = [var for var in train_X.columns if train_X[var].dtype == 'O']

In [ ]:
def find_frequent_labels(df, var, rare_perc):
    
    # function finds the labels that are shared by more than
    # a certain % of the houses in the dataset
    df = df.copy()
    tmp = df.groupby(var)['SalePrice'].count() / len(df)
    return tmp[tmp > rare_perc].index


for var in cat_vars:
    
    # find the frequent categories
    frequent_ls = find_frequent_labels(train_X, var, 0.01)
    
    # replace rare categories by the string "Rare"
    train_X[var] = np.where(train_X[var].isin(
        frequent_ls), train_X[var], 'Rare')
    
    val_X[var] = np.where(val_X[var].isin(
        frequent_ls), val_X[var], 'Rare')

##### Encoding of categorical variables

Next, I need to transform the strings of the categorical variables into numbers. I do this to capture the monotonic relationship between the label and the target.

This function will assign discrete values to the strings of the variables, so that the smaller value corresponds to the category that shows the smaller mean house sale price.

In [ ]:
def replace_categories(train, test, var, target):

    # order the categories in a variable from that with the lowest
    # house sale price, to that with the highest
    ordered_labels = train.groupby([var])[target].mean().sort_values().index

    # create a dictionary of ordered categories to integer values
    ordinal_label = {k: i for i, k in enumerate(ordered_labels, 0)}

    # use the dictionary to replace the categorical strings by integers
    train[var] = train[var].map(ordinal_label)
    test[var] = test[var].map(ordinal_label)

In [ ]:
for var in cat_vars:
    replace_categories(train_X, val_X, var, 'SalePrice')

In [ ]:
# check absence of na in the train set
[var for var in train_X.columns if train_X[var].isnull().sum() > 0]

In [ ]:
# check absence of na in the test set
[var for var in val_X.columns if val_X[var].isnull().sum() > 0]

In [ ]:
# let me show you what I mean by monotonic relationship
# between labels and target

def analyse_vars(df, var):
    
    # function plots median house sale price per encoded
    # category
    
    df = df.copy()
    df.groupby(var)['SalePrice'].median().plot.bar()
    plt.title(var)
    plt.ylabel('SalePrice')
    plt.show()
    
for var in cat_vars:
    analyse_vars(train_X, var)

#### Feature Scaling

In [ ]:
# capture all variables in a list
# except the target and the ID

train_vars = [var for var in train_X.columns if var not in ['Id', 'SalePrice']]

# count number of variables
len(train_vars)

In [ ]:
# feature scaling
from sklearn.preprocessing import MinMaxScaler

# create scaler
scaler = MinMaxScaler()

#  fit  the scaler to the train set
scaler.fit(train_X[train_vars]) 

# transform the train and test set
train_X[train_vars] = scaler.transform(train_X[train_vars])

val_X[train_vars] = scaler.transform(val_X[train_vars])

In [ ]:
train_X.head()

#### Feature Selection

I did the model fitting and feature selection altogether in a few lines of code. First, I specified the Lasso Regression model, and we select a suitable alpha (equivalent of penalty). The bigger the alpha the less features that will be selected.

Then I used the selectFromModel object from sklearn, which will select automatically the features which coefficients are non-zero.

In [ ]:
# to build the models
from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel

# remember to set the seed, the random state in this function
sel_ = SelectFromModel(Lasso(alpha=0.005, random_state=0))

# train Lasso model and select features
sel_.fit(train_X, train_y)

Let's visualise those features that were selected (where selected features are True).

In [ ]:
sel_.get_support()

Let's print the number of total and selected features; this is how we can make a list of the selected features.

In [ ]:
selected_feats = train_X.columns[(sel_.get_support())]

# let's print some stats
print('total features: {}'.format((train_X.shape[1])))
print('selected features: {}'.format(len(selected_feats)))
print('features with coefficients shrank to zero: {}'.format(
    np.sum(sel_.estimator_.coef_ == 0)))

In [ ]:
# print the selected features
selected_feats

### Step 3. Score a model

In [ ]:
# load the train and test set with the engineered variables
X_full = pd.read_csv('train.csv')

In [ ]:
train_X

#### Random Forest Regressions

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Define the models
model_1 = RandomForestRegressor(n_estimators=50, random_state=0)
model_2 = RandomForestRegressor(n_estimators=100, random_state=0)
model_3 = RandomForestRegressor(n_estimators=100, criterion='mae', random_state=0)
model_4 = RandomForestRegressor(n_estimators=200, min_samples_split=20, random_state=0)
model_5 = RandomForestRegressor(n_estimators=100, max_depth=7, random_state=0)

models = [model_1, model_2, model_3, model_4, model_5]

In [ ]:
from sklearn.metrics import mean_absolute_error

# Function for comparing different models
def score_model(model, X_t=train_X, X_v=val_X, y_t=train_y, y_v=val_y):
    model.fit(X_t, y_t)
    preds = model.predict(X_v)
    return mean_absolute_error(y_v, preds)

for i in range(0, len(models)):
    mae = score_model(models[i])
    print("Model %d MAE: %d" % (i+1, mae))

##### Mean Absolute Error 

The goal is to see the difference in mean absolute errors (mae) each using different arguments and parameters. In Model 1, we observe the lowest n-estimate at 50 and the highest n-estimate at 200 for model 4. The only criterion equal to mae result in the lowest score, which means this calculates how good the train_test_split is. Whether we use the minimum split or max depth in both model 4 and 5, it will not result in any improvement in model performance.   

#### Scoring Dataset

In [ ]:
# Remove rows with missing target, separate target from predictors
X_full.dropna(axis=0, subset=['SalePrice'], inplace=True)
y = X_full.SalePrice
X_full.drop(['SalePrice'], axis=1, inplace=True)

# To keep things simple, we'll use only numerical predictors
X = X_full.select_dtypes(exclude=['object'])

In [ ]:
# Shape of training data (num_rows, num_columns)
print(train_X.shape)

# Number of missing values in each column of training data
missing_val_count_by_column = (train_X.isnull().sum())
print(missing_val_count_by_column[missing_val_count_by_column > 0])

We removed rows from the missing target Sale Price in the train_X set with all the engineered features. The dependent variable will serve as Sale Price in the train_y set and this meant we dropped the Sale Price from the train_X set.

Using only numerical predictors meant that we did not have to worry about the integers and float characters. We can see that the number of missing values is great where 1095 rows and 7 columns have missing values we just omitted. This will help the performance of our model. 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error

# Function for comparing different approaches
def score_dataset(train_X, val_X, train_y, val_y):
    model = RandomForestRegressor(n_estimators=100, random_state=0)
    model.fit(train_X, train_y)
    preds = model.predict(val_X)
    return mean_absolute_error(val_y, preds)

In [ ]:
# Score average MAE in the training data. 
score_dataset(train_X, val_X, train_y, val_y)

The average MAE, using the best random forest model, returns a value close to model 2 as it is the best fit. 

### Step 4. Build a Model

### Evaluate the model

In [ ]:
train_X.isna().sum()
train_y.isna().sum()

#### Regularised linear regression: Lasso

In [ ]:
# to build the model
from sklearn.linear_model import Lasso, LassoCV

# set up the model
# remember to set the random_state / seed
lin_model = Lasso(alpha=0.005, random_state=0)

# train the model
lin_model.fit(train_X, train_y)

In [ ]:
# make predictions for train set
pred = lin_model.predict(train_X)

#### Remember that we log transformed the output (SalePrice) in our feature engineering step. In order to get the true performance of the Lasso, I need to transform both the target and the predictions back to the original house prices values. 

Here, I evaluate performance using the mean squared error and the root of the mean squared error.

In [ ]:
# to evaluate the model
from sklearn.metrics import mean_squared_error
from math import sqrt

# determine mse and rmse
print('train mse: {}'.format(int(
    mean_squared_error(train_y, lin_model.predict(train_X)))))
print('train rmse: {}'.format(int(
    sqrt(mean_squared_error(val_y, lin_model.predict(val_X))))))
print()

In [ ]:
# let's evaluate our predictions respect to the real sale price
plt.scatter(val_y, lin_model.predict(val_X))
plt.xlabel('True House Price')
plt.ylabel('Predicted House Price')
plt.title('Evaluation of Lasso Predictions')

We can see that our model is doing a pretty good job at estimating house prices.

Let's evaluate the distribution of the errors: they should be fairly normally distributed.

In [ ]:
errors = val_y - lin_model.predict(val_X)
errors.hist(bins=50)

The distribution of the errors follows closely a gaussian distribution. Again, that suggests that our model is doing a good job. 

In [ ]:
# Let's look at the feature importance
importance = pd.Series(np.abs(lin_model.coef_.ravel()))
importance.index = selected_feats
importance.sort_values(inplace=True, ascending=False)
importance.plot.bar(figsize=(18,6))
plt.ylabel('Lasso Coefficients')
plt.title('Feature Importance')